## FashionMNIST DataSet & DataLoader

In [59]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [60]:
train_data = datasets.FashionMNIST(
    root="../../data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../../data", train=False, download=True, transform=ToTensor()
)

In [61]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

## Modeling

In [62]:
from torch import nn


class NeuralNetwork(nn.Module):
    def __init__(self):
        # nn.Module 생성
        super(NeuralNetwork, self).__init__()

        # 레이어 정의
        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 128), nn.ReLU(), nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        y = self.linear_relu_stack(x)

        return y

## 모델 생성

In [63]:
import torch

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device = "cpu"

print(device)

cpu


In [64]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


## Train Model

In [65]:
loss_fn = nn.CrossEntropyLoss()  # 이미 여기에 소프트맥스 함수가 포함되어 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [66]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # DataLoader 에 들어있는 tensor 들을 모델과 같은 위치에 두기
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()  # 이전 배치의 기울기 제거
        loss.backward()
        optimizer.step()

        # 배치가 100번 돌 때마다 화면에 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

In [67]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    # loss는 배치 별로 계산, correct는 전체 데이터 세트에 대한 평균 정확도
    test_loss, correct = 0, 0

    # 모델을 추론 모드로 바꿔준다.
    model.eval()

    # 추론 과정에서는 기울기를 구할 필요 없음
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)

            test_loss += loss_fn(pred, y).item()

            # 10개의 예측값 중 가장 큰 곳의 인덱스를 argmax 로 찾고, target 과 일치하는지 확인
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    num_batches = len(dataloader)
    test_loss /= num_batches
    correct /= size

    print(
        f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n"
    )

In [68]:
epochs = 10

for i in range(epochs):
    print(f"Epoch {i+1}\n........................")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Experiment Successful")

Epoch 1
........................
Train Loss : 2.297895 [     0 / 60000 ]
Train Loss : 1.478404 [  6400 / 60000 ]
Train Loss : 1.130491 [ 12800 / 60000 ]
Train Loss : 0.929421 [ 19200 / 60000 ]
Train Loss : 0.704111 [ 25600 / 60000 ]
Train Loss : 0.659142 [ 32000 / 60000 ]
Train Loss : 0.603530 [ 38400 / 60000 ]
Train Loss : 0.589859 [ 44800 / 60000 ]
Train Loss : 0.773566 [ 51200 / 60000 ]
Train Loss : 0.721782 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 79.0%, Avg Loss : 0.622938

Epoch 2
........................
Train Loss : 0.687222 [     0 / 60000 ]
Train Loss : 0.608836 [  6400 / 60000 ]
Train Loss : 0.506180 [ 12800 / 60000 ]
Train Loss : 0.580014 [ 19200 / 60000 ]
Train Loss : 0.606995 [ 25600 / 60000 ]
Train Loss : 0.436001 [ 32000 / 60000 ]
Train Loss : 0.490737 [ 38400 / 60000 ]
Train Loss : 0.566170 [ 44800 / 60000 ]
Train Loss : 0.741702 [ 51200 / 60000 ]
Train Loss : 0.549720 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 81.9%, Avg Loss : 0.530969

Epoch 3
..................

## 훈련된 모델의 가중치를 저장 / 불러오기

불러올 곳에서 모델의 구조를 알고 있는 경우 가중치만 저장하면 적은 용량으로 저장하고 불러오는 것이 가능하다
* 단점 : 모델 class 가 없으면 사용할 수 없다

In [69]:
# model.state_dict() : 모델 내에 있는 레이어 별 가중치를 들고 있는 딕셔너리
torch.save(model.state_dict(), "../../data/saved_models/model_weights.pth")

In [70]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-0.0355,  0.0576,  0.0760,  ...,  0.0381, -0.0149, -0.0047],
                      [ 0.0018,  0.0540,  0.0017,  ..., -0.0242, -0.0169, -0.0443],
                      [-0.0125,  0.0359,  0.0855,  ...,  0.0512,  0.0311,  0.0280],
                      ...,
                      [ 0.0023, -0.0212,  0.0302,  ...,  0.0632,  0.0274,  0.0362],
                      [ 0.0112,  0.0137, -0.0035,  ..., -0.0926, -0.0960, -0.0515],
                      [-0.0097, -0.0538, -0.0091,  ...,  0.0126, -0.0013, -0.0419]])),
             ('linear_relu_stack.0.bias',
              tensor([ 0.1641,  0.1191,  0.1945,  0.1843,  0.1274,  0.0384,  0.1799,  0.1182,
                      -0.0395,  0.1323, -0.0072,  0.0969, -0.1272,  0.2525,  0.0588,  0.0718,
                       0.1901,  0.0257,  0.1544,  0.0404,  0.0228,  0.0440, -0.0922,  0.0752,
                      -0.0298,  0.0786, -0.0384,  0.0220,  0.0576,  0.1186,  0.1308,  0.0792,
    

In [71]:
# 저장된 가중치 파일(pth) 불러오기
new_model = NeuralNetwork().to(device)
print(new_model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [72]:
# 훈련되지 않은 모델로 예측
test_loop(test_dataloader, new_model, loss_fn)

Test Error : 
 Accuracy : 11.4%, Avg Loss : 2.314730



In [73]:
# 이전에 훈련된 가중치를 불러와 새로운 모델에 load
new_model.load_state_dict(torch.load("../../data/saved_models/model_weights.pth"))
test_loop(test_dataloader, new_model, loss_fn)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_98219/3047005004.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model.load_state_dict(torch.load("../../d

Test Error : 
 Accuracy : 85.2%, Avg Loss : 0.415099



## 훈련된 모델 자체를 저장 / 불러오기

* 모델의 구조를 몰라도 사용 가능
* 용량은 많이 필요하다

In [74]:
torch.save(model, "../../data/saved_models/model.pth")

In [75]:
!ls -al | grep pth

In [76]:
new_model = torch.load("../../data/saved_models/model.pth")
test_loop(test_dataloader, new_model, loss_fn)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_98219/2093080944.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model = torch.load("../../data/saved_mode

Test Error : 
 Accuracy : 85.2%, Avg Loss : 0.415099

